## Google Distance Matrix Trial
Tested Google Distance Matrix based on 10 O-D pairs with different departure times(durtion, duration_in_traffic).

In [1]:
import requests
import googlemaps
import datetime
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import itertools
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import json
import folium
import polyline
from shapely import wkt
import time

/Users/kweon-youngseo/miniforge3/envs/ppd599/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/var/folders/nh/bgd128891hxd7ggprg1zgbhm0000gn/T/ipykernel_98996/859036864.py:5: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Sh

In [2]:
# Jessica Google API Key
key = 'Jessica Google API Key'

In [3]:
origins_df = pd.read_csv("../Data/origins_gdf.csv")
destinations_df = pd.read_csv("../Data/destinations_gdf.csv")

In [4]:
origins_df['geometry'] = origins_df['geometry'].apply(wkt.loads)
destinations_df['geometry'] = destinations_df['geometry'].apply(wkt.loads)

In [5]:
origins_gdf = gpd.GeoDataFrame(origins_df, geometry='geometry')
destinations_gdf = gpd.GeoDataFrame(destinations_df, geometry='geometry')

In [6]:
type(destinations_gdf)

geopandas.geodataframe.GeoDataFrame

## Calculate Free-Flow Travel Time

In [7]:
gmaps = googlemaps.Client(key=key)

In [8]:
# extract origin and destination coordinates from the GeoDataFrames
origins = origins_gdf.geometry.apply(lambda point: (point.y, point.x)).tolist()
destinations = destinations_gdf.geometry.apply(lambda point: (point.y, point.x)).tolist()

In [9]:
# trial at 9/10 6:35 pm
result = gmaps.distance_matrix(origins, destinations, mode="driving")

In [10]:
result

{'destination_addresses': ['7817 S Central Ave, Los Angeles, CA 90001, USA',
  '1818 W Mountain St, Glendale, CA 91201, USA',
  'Edgemont & Lexington Ave., Los Angeles, CA 90029, USA',
  '412 Conway Ct, Glendora, CA 91741, USA',
  '1973 241st St, Lomita, CA 90717, USA',
  '4506 Santa Rosalia Dr, Los Angeles, CA 90008, USA',
  '8967 Shoreham Dr, Los Angeles, CA 90069, USA',
  'Llano Cutoff, Llano, CA 93544, USA',
  '14703 S Castlegate Ave, Compton, CA 90221, USA',
  '891 W 2nd St, Pomona, CA 91766, USA'],
 'origin_addresses': ['654 W Elberon Ave, San Pedro, CA 90731, USA',
  '1366 W 13th St, San Pedro, CA 90732, USA',
  '17053 Mindora Ct, Granada Hills, CA 91344, USA',
  '3636 Ponderosa Way, Palmdale, CA 93550, USA',
  '17508 Ludlow St, Granada Hills, CA 91344, USA',
  '6525 N Sepulveda Blvd, Van Nuys, CA 91406, USA',
  '13627 Longworth Ave, Norwalk, CA 90650, USA',
  '3743 Fredonia Dr, Los Angeles, CA 90068, USA',
  '5402 Chesebro Rd, Agoura Hills, CA 91301, USA',
  '332 W Meda Ave, Gl

In [59]:
# create free_flow travel time (at 9/8 10:48 pm)
free_flow_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    free_flow_times.append(travel_time)

In [60]:
free_flow_times

['24 mins',
 '50 mins',
 '27 mins',
 '1 hour 16 mins',
 '53 mins',
 '29 mins',
 '49 mins',
 '1 hour 11 mins',
 '1 hour 1 min',
 '19 mins']

In [29]:
# create free_flow travel time (at 9/10 7/52 pm)
free_flow_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    free_flow_times.append(travel_time)

In [30]:
free_flow_times

['24 mins',
 '50 mins',
 '27 mins',
 '1 hour 16 mins',
 '49 mins',
 '29 mins',
 '51 mins',
 '1 hour 11 mins',
 '57 mins',
 '19 mins']

In [48]:
# create distance
distances = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving")

    # extract distance from the result
    distance = result['rows'][0]['elements'][0]['distance']['text']
    distances.append(distance)


In [49]:
distances

['27.9 km',
 '57.6 km',
 '33.5 km',
 '126 km',
 '63.8 km',
 '31.0 km',
 '48.4 km',
 '108 km',
 '77.3 km',
 '19.5 km']

## Real-Time Travel Time

In [21]:
# trial at 9/10 6:35 pm
result = gmaps.distance_matrix(origins[0], destinations[0], mode="driving", departure_time='now')
result

{'destination_addresses': ['7817 S Central Ave, Los Angeles, CA 90001, USA'],
 'origin_addresses': ['654 W Elberon Ave, San Pedro, CA 90731, USA'],
 'rows': [{'elements': [{'distance': {'text': '27.9 km', 'value': 27865},
     'duration': {'text': '24 mins', 'value': 1466},
     'duration_in_traffic': {'text': '24 mins', 'value': 1467},
     'status': 'OK'}]}],
 'status': 'OK'}

In [61]:
# create free_flow travel time (at 10:48 pm)
real_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    real_times.append(travel_time)

In [ ]:
real_times

In [22]:
# create free_flow travel time (at 9/10 7:48 pm)
real_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract travel time from the result
    travel_time = result['rows'][0]['elements'][0]['duration']['text']
    real_times.append(travel_time)

In [23]:
real_times

['24 mins',
 '50 mins',
 '27 mins',
 '1 hour 16 mins',
 '49 mins',
 '29 mins',
 '50 mins',
 '1 hour 11 mins',
 '54 mins',
 '19 mins']

In [27]:
real_times = pd.DataFrame(real_times)
real_times

,0
0,24 mins
1,50 mins
2,27 mins
3,1 hour 16 mins
4,49 mins
5,29 mins
6,50 mins
7,1 hour 11 mins
8,54 mins
9,19 mins


In [24]:
# create travel time (at 9/10 7:48 pm)
traffic_times = []
for origin, destination in zip(origins, destinations):
    # call the Distance Matrix API
    result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time='now')

    # extract duration_in_traffic from the result
    travel_time = result['rows'][0]['elements'][0]['duration_in_traffic']['text']
    traffic_times.append(travel_time)

In [25]:
traffic_times

['25 mins',
 '52 mins',
 '26 mins',
 '1 hour 15 mins',
 '49 mins',
 '28 mins',
 '46 mins',
 '1 hour 11 mins',
 '58 mins',
 '17 mins']

In [28]:
traffic_times = pd.DataFrame(traffic_times)
traffic_times

,0
0,25 mins
1,52 mins
2,26 mins
3,1 hour 15 mins
4,49 mins
5,28 mins
6,46 mins
7,1 hour 11 mins
8,58 mins
9,17 mins


## Organize Data

In [67]:
origins_df = origins_df.columns = ["origins_" + col for col in origins_df.columns]
destinations_df = destinations_df.columns = ["destinations_" + col for col in destinations_df.columns]

In [69]:
combined_df = pd.concat([origins_df, destinations_df], axis=1)

In [70]:
combined_df

,origins_u,origins_v,origins_key,origins_geometry,origins_address,destinations_u,destinations_v,destinations_key,destinations_geometry,destinations_address
0,298221680,1931344922,0,POINT (-118.2906052737158 33.748931966843635),"CA 47, Los Angeles, Los Angeles County, Califo...",1774095082,1845269311,0,POINT (-118.25640865158168 33.968032359194574),"7798, South Central Avenue, Florence, Los Ange..."
1,122623196,122819932,0,POINT (-118.3071916244818 33.73247904045904),"1370, West 13th Street, San Pedro, Los Angeles...",355623372,122752417,0,POINT (-118.27657401374343 34.18182136334436),"Glendale, Los Angeles County, California, 9120..."
2,123763751,122693938,0,POINT (-118.50403533012181 34.289628248401165),"17001, Mindora Court, Granada Hills North Neig...",123259961,123526962,0,POINT (-118.29611306201004 34.093428307290225),"4901, Lexington Avenue, East Hollywood, Los An..."
3,4726456954,122784052,0,POINT (-118.06502087503895 34.542724152710036),"3625, Pearblossom Highway, Palmdale, Los Angel...",10820069648,123076302,0,POINT (-117.88499521527298 34.14159336855743),"18693, East Milton Drive, La Fetra, Glendora, ..."
4,123257162,123257159,0,POINT (-118.51592473996995 34.27092641376895),"17525, Ludlow Street, Granada Hills, Granada H...",123657337,123176647,0,POINT (-118.31418854269856 33.806307946302326),"1999, 241st Street, Lomita, Los Angeles County..."
5,370386634,370386179,0,POINT (-118.46642989917375 34.18946956290023),"6552, Sepulveda Boulevard, Van Nuys Neighborho...",122850436,122850438,0,POINT (-118.34723944167807 34.01506564736644),"3900, Potomac Avenue, Crenshaw, Los Angeles, L..."
6,122593405,123382229,0,POINT (-118.09311124775975 33.90729395332569),"13673, Longworth Avenue, Norwalk, Los Angeles ...",122586184,123641220,0,POINT (-118.38713973841537 34.09142212002091),"8901, Shoreham Drive, Los Angeles, Los Angeles..."
7,408485712,1525398879,0,POINT (-118.36300794186137 34.13322903583341),"3745, Fredonia Drive, Los Angeles, Los Angeles...",123071143,123309236,0,POINT (-117.80895137386311 34.488648955968806),"Llano Ctf, Los Angeles County, California, 935..."
8,123041230,123509383,0,POINT (-118.73718089273797 34.148200635247264),"5449, Chesebro Road, Agoura, Agoura Hills, Los...",2546485056,122622198,0,POINT (-118.19700344686923 33.89942758940711),"14636, East San Vincente Street, East Compton,..."
9,3749699449,122812666,0,POINT (-117.86926576935714 34.138516431108876),"332, West Meda Avenue, Glendora, Los Angeles C...",123399006,122858265,0,POINT (-117.75890605167689 34.0576222887382),"901, West 2nd Street, Edison Historic District..."


In [71]:
combined_df['free_flow_time_10:48pm'] = free_flow_times

In [73]:
combined_df['real_times_10:48pm'] = real_times

In [74]:
combined_df

,origins_u,origins_v,origins_key,origins_geometry,origins_address,destinations_u,destinations_v,destinations_key,destinations_geometry,destinations_address,free_flow_time_10:48pm,real_times_10:48pm
0,298221680,1931344922,0,POINT (-118.2906052737158 33.748931966843635),"CA 47, Los Angeles, Los Angeles County, Califo...",1774095082,1845269311,0,POINT (-118.25640865158168 33.968032359194574),"7798, South Central Avenue, Florence, Los Ange...",24 mins,24 mins
1,122623196,122819932,0,POINT (-118.3071916244818 33.73247904045904),"1370, West 13th Street, San Pedro, Los Angeles...",355623372,122752417,0,POINT (-118.27657401374343 34.18182136334436),"Glendale, Los Angeles County, California, 9120...",50 mins,50 mins
2,123763751,122693938,0,POINT (-118.50403533012181 34.289628248401165),"17001, Mindora Court, Granada Hills North Neig...",123259961,123526962,0,POINT (-118.29611306201004 34.093428307290225),"4901, Lexington Avenue, East Hollywood, Los An...",27 mins,28 mins
3,4726456954,122784052,0,POINT (-118.06502087503895 34.542724152710036),"3625, Pearblossom Highway, Palmdale, Los Angel...",10820069648,123076302,0,POINT (-117.88499521527298 34.14159336855743),"18693, East Milton Drive, La Fetra, Glendora, ...",1 hour 16 mins,1 hour 20 mins
4,123257162,123257159,0,POINT (-118.51592473996995 34.27092641376895),"17525, Ludlow Street, Granada Hills, Granada H...",123657337,123176647,0,POINT (-118.31418854269856 33.806307946302326),"1999, 241st Street, Lomita, Los Angeles County...",53 mins,53 mins
5,370386634,370386179,0,POINT (-118.46642989917375 34.18946956290023),"6552, Sepulveda Boulevard, Van Nuys Neighborho...",122850436,122850438,0,POINT (-118.34723944167807 34.01506564736644),"3900, Potomac Avenue, Crenshaw, Los Angeles, L...",29 mins,28 mins
6,122593405,123382229,0,POINT (-118.09311124775975 33.90729395332569),"13673, Longworth Avenue, Norwalk, Los Angeles ...",122586184,123641220,0,POINT (-118.38713973841537 34.09142212002091),"8901, Shoreham Drive, Los Angeles, Los Angeles...",49 mins,54 mins
7,408485712,1525398879,0,POINT (-118.36300794186137 34.13322903583341),"3745, Fredonia Drive, Los Angeles, Los Angeles...",123071143,123309236,0,POINT (-117.80895137386311 34.488648955968806),"Llano Ctf, Los Angeles County, California, 935...",1 hour 11 mins,1 hour 11 mins
8,123041230,123509383,0,POINT (-118.73718089273797 34.148200635247264),"5449, Chesebro Road, Agoura, Agoura Hills, Los...",2546485056,122622198,0,POINT (-118.19700344686923 33.89942758940711),"14636, East San Vincente Street, East Compton,...",1 hour 1 min,59 mins
9,3749699449,122812666,0,POINT (-117.86926576935714 34.138516431108876),"332, West Meda Avenue, Glendora, Los Angeles C...",123399006,122858265,0,POINT (-117.75890605167689 34.0576222887382),"901, West 2nd Street, Edison Historic District...",19 mins,19 mins


## Set Times

In [97]:
# trial

departure_time_obj = datetime.datetime(2023, 10, 1, 11)
departure_time_unix = int(time.mktime(departure_time_obj.timetuple()))

result = gmaps.distance_matrix(origins=origins[0], destinations=destinations[0], mode="driving", departure_time=departure_time_unix)

result

{'destination_addresses': ['7817 S Central Ave, Los Angeles, CA 90001, USA'],
 'origin_addresses': ['654 W Elberon Ave, San Pedro, CA 90731, USA'],
 'rows': [{'elements': [{'distance': {'text': '27.9 km', 'value': 27865},
     'duration': {'text': '24 mins', 'value': 1466},
     'duration_in_traffic': {'text': '24 mins', 'value': 1426},
     'status': 'OK'}]}],
 'status': 'OK'}

In [75]:
# set departure_times
start_time = datetime.datetime(2023, 10, 1)
departure_times = [start_time + datetime.timedelta(hours=i) for i in range(24)]

In [88]:
travel_times = []

# calculate the travel time for each pair
for i in range(10): 
    origin = origins[i]
    destination = destinations[i]
    
    travel_times_by_hour = {}

    for dt in departure_times:
        hour_str = dt.strftime('%H')
        column_name = f'duration_{hour_str}'
        
        result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time=dt)
        
        travel_time = result['rows'][0]['elements'][0]['duration']['text']
        travel_times_by_hour[column_name] = travel_time
    
    travel_times.append(travel_times_by_hour)

In [89]:
travel_times_df = pd.DataFrame(travel_times)

In [90]:
travel_times_df

,duration_00,duration_01,duration_02,duration_03,duration_04,duration_05,duration_06,duration_07,duration_08,duration_09,...,duration_14,duration_15,duration_16,duration_17,duration_18,duration_19,duration_20,duration_21,duration_22,duration_23
0,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,...,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins,24 mins
1,50 mins,52 mins,50 mins,50 mins,50 mins,50 mins,52 mins,50 mins,50 mins,50 mins,...,50 mins,50 mins,50 mins,50 mins,50 mins,50 mins,50 mins,50 mins,50 mins,50 mins
2,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,...,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins,27 mins
3,1 hour 16 mins,1 hour 20 mins,1 hour 20 mins,1 hour 20 mins,1 hour 20 mins,1 hour 20 mins,1 hour 16 mins,1 hour 20 mins,1 hour 20 mins,1 hour 16 mins,...,1 hour 16 mins,1 hour 16 mins,1 hour 16 mins,1 hour 16 mins,1 hour 16 mins,1 hour 16 mins,1 hour 20 mins,1 hour 20 mins,1 hour 20 mins,1 hour 16 mins
4,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,...,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins
5,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,...,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins,28 mins
6,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,...,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins,49 mins
7,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,...,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins,1 hour 11 mins
8,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,...,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins,54 mins
9,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,...,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins,19 mins


In [91]:
travel_times_t = []

# calculate the travel time for each pair
for i in range(10):
    origin = origins[i]
    destination = destinations[i]
    
    travel_times_by_hour = {}

    for dt in departure_times:
        hour_str = dt.strftime('%H')
        column_name = f'duration_in_traffic_{hour_str}'
        
        result = gmaps.distance_matrix(origins=origin, destinations=destination, mode="driving", departure_time=dt)
        
        travel_time = result['rows'][0]['elements'][0]['duration_in_traffic']['text']
        travel_times_by_hour[column_name] = travel_time
    
    travel_times_t.append(travel_times_by_hour)

In [92]:
travel_times_df2 = pd.DataFrame(travel_times_t)

In [93]:
travel_times_df2

,duration_in_traffic_00,duration_in_traffic_01,duration_in_traffic_02,duration_in_traffic_03,duration_in_traffic_04,duration_in_traffic_05,duration_in_traffic_06,duration_in_traffic_07,duration_in_traffic_08,duration_in_traffic_09,...,duration_in_traffic_14,duration_in_traffic_15,duration_in_traffic_16,duration_in_traffic_17,duration_in_traffic_18,duration_in_traffic_19,duration_in_traffic_20,duration_in_traffic_21,duration_in_traffic_22,duration_in_traffic_23
0,22 mins,22 mins,21 mins,21 mins,21 mins,21 mins,21 mins,21 mins,22 mins,22 mins,...,25 mins,25 mins,24 mins,24 mins,24 mins,23 mins,24 mins,24 mins,22 mins,22 mins
1,43 mins,41 mins,40 mins,40 mins,41 mins,40 mins,40 mins,41 mins,41 mins,43 mins,...,58 mins,57 mins,55 mins,52 mins,49 mins,46 mins,47 mins,47 mins,43 mins,42 mins
2,24 mins,24 mins,24 mins,24 mins,23 mins,23 mins,22 mins,23 mins,23 mins,24 mins,...,26 mins,26 mins,26 mins,26 mins,26 mins,25 mins,26 mins,26 mins,25 mins,24 mins
3,1 hour 12 mins,1 hour 12 mins,1 hour 12 mins,1 hour 11 mins,1 hour 11 mins,1 hour 10 mins,1 hour 8 mins,1 hour 11 mins,1 hour 10 mins,1 hour 11 mins,...,1 hour 17 mins,1 hour 16 mins,1 hour 15 mins,1 hour 13 mins,1 hour 14 mins,1 hour 12 mins,1 hour 15 mins,1 hour 14 mins,1 hour 12 mins,1 hour 11 mins
4,44 mins,42 mins,42 mins,42 mins,42 mins,42 mins,41 mins,42 mins,43 mins,44 mins,...,1 hour 1 min,59 mins,56 mins,52 mins,49 mins,47 mins,48 mins,47 mins,44 mins,43 mins
5,24 mins,23 mins,23 mins,23 mins,23 mins,23 mins,22 mins,23 mins,24 mins,24 mins,...,36 mins,35 mins,33 mins,31 mins,30 mins,27 mins,27 mins,27 mins,25 mins,24 mins
6,38 mins,36 mins,35 mins,34 mins,34 mins,33 mins,34 mins,34 mins,36 mins,38 mins,...,57 mins,54 mins,52 mins,50 mins,48 mins,44 mins,44 mins,44 mins,40 mins,37 mins
7,1 hour 7 mins,1 hour 6 mins,1 hour 6 mins,1 hour 6 mins,1 hour 6 mins,1 hour 6 mins,1 hour 5 mins,1 hour 5 mins,1 hour 6 mins,1 hour 7 mins,...,1 hour 10 mins,1 hour 9 mins,1 hour 9 mins,1 hour 8 mins,1 hour 7 mins,1 hour 7 mins,1 hour 9 mins,1 hour 9 mins,1 hour 7 mins,1 hour 6 mins
8,49 mins,47 mins,47 mins,48 mins,47 mins,47 mins,46 mins,46 mins,47 mins,48 mins,...,1 hour 8 mins,1 hour 6 mins,1 hour 3 mins,1 hour 1 min,59 mins,56 mins,58 mins,54 mins,54 mins,49 mins
9,16 mins,16 mins,16 mins,16 mins,16 mins,16 mins,15 mins,16 mins,16 mins,16 mins,...,18 mins,17 mins,17 mins,17 mins,17 mins,17 mins,18 mins,17 mins,17 mins,17 mins


## Organize Data

In [98]:
combined_df = pd.concat([combined_df, travel_times_df], axis=1)

In [99]:
combined_df = pd.concat([combined_df, travel_times_df2], axis=1)

In [103]:
combined_df['distance'] = distances

In [104]:
combined_df

,origins_u,origins_v,origins_key,origins_geometry,origins_address,destinations_u,destinations_v,destinations_key,destinations_geometry,destinations_address,...,duration_in_traffic_15,duration_in_traffic_16,duration_in_traffic_17,duration_in_traffic_18,duration_in_traffic_19,duration_in_traffic_20,duration_in_traffic_21,duration_in_traffic_22,duration_in_traffic_23,distance
0,298221680,1931344922,0,POINT (-118.2906052737158 33.748931966843635),"CA 47, Los Angeles, Los Angeles County, Califo...",1774095082,1845269311,0,POINT (-118.25640865158168 33.968032359194574),"7798, South Central Avenue, Florence, Los Ange...",...,25 mins,24 mins,24 mins,24 mins,23 mins,24 mins,24 mins,22 mins,22 mins,27.9 km
1,122623196,122819932,0,POINT (-118.3071916244818 33.73247904045904),"1370, West 13th Street, San Pedro, Los Angeles...",355623372,122752417,0,POINT (-118.27657401374343 34.18182136334436),"Glendale, Los Angeles County, California, 9120...",...,57 mins,55 mins,52 mins,49 mins,46 mins,47 mins,47 mins,43 mins,42 mins,57.6 km
2,123763751,122693938,0,POINT (-118.50403533012181 34.289628248401165),"17001, Mindora Court, Granada Hills North Neig...",123259961,123526962,0,POINT (-118.29611306201004 34.093428307290225),"4901, Lexington Avenue, East Hollywood, Los An...",...,26 mins,26 mins,26 mins,26 mins,25 mins,26 mins,26 mins,25 mins,24 mins,33.5 km
3,4726456954,122784052,0,POINT (-118.06502087503895 34.542724152710036),"3625, Pearblossom Highway, Palmdale, Los Angel...",10820069648,123076302,0,POINT (-117.88499521527298 34.14159336855743),"18693, East Milton Drive, La Fetra, Glendora, ...",...,1 hour 16 mins,1 hour 15 mins,1 hour 13 mins,1 hour 14 mins,1 hour 12 mins,1 hour 15 mins,1 hour 14 mins,1 hour 12 mins,1 hour 11 mins,126 km
4,123257162,123257159,0,POINT (-118.51592473996995 34.27092641376895),"17525, Ludlow Street, Granada Hills, Granada H...",123657337,123176647,0,POINT (-118.31418854269856 33.806307946302326),"1999, 241st Street, Lomita, Los Angeles County...",...,59 mins,56 mins,52 mins,49 mins,47 mins,48 mins,47 mins,44 mins,43 mins,63.8 km
5,370386634,370386179,0,POINT (-118.46642989917375 34.18946956290023),"6552, Sepulveda Boulevard, Van Nuys Neighborho...",122850436,122850438,0,POINT (-118.34723944167807 34.01506564736644),"3900, Potomac Avenue, Crenshaw, Los Angeles, L...",...,35 mins,33 mins,31 mins,30 mins,27 mins,27 mins,27 mins,25 mins,24 mins,31.0 km
6,122593405,123382229,0,POINT (-118.09311124775975 33.90729395332569),"13673, Longworth Avenue, Norwalk, Los Angeles ...",122586184,123641220,0,POINT (-118.38713973841537 34.09142212002091),"8901, Shoreham Drive, Los Angeles, Los Angeles...",...,54 mins,52 mins,50 mins,48 mins,44 mins,44 mins,44 mins,40 mins,37 mins,48.4 km
7,408485712,1525398879,0,POINT (-118.36300794186137 34.13322903583341),"3745, Fredonia Drive, Los Angeles, Los Angeles...",123071143,123309236,0,POINT (-117.80895137386311 34.488648955968806),"Llano Ctf, Los Angeles County, California, 935...",...,1 hour 9 mins,1 hour 9 mins,1 hour 8 mins,1 hour 7 mins,1 hour 7 mins,1 hour 9 mins,1 hour 9 mins,1 hour 7 mins,1 hour 6 mins,108 km
8,123041230,123509383,0,POINT (-118.73718089273797 34.148200635247264),"5449, Chesebro Road, Agoura, Agoura Hills, Los...",2546485056,122622198,0,POINT (-118.19700344686923 33.89942758940711),"14636, East San Vincente Street, East Compton,...",...,1 hour 6 mins,1 hour 3 mins,1 hour 1 min,59 mins,56 mins,58 mins,54 mins,54 mins,49 mins,77.3 km
9,3749699449,122812666,0,POINT (-117.86926576935714 34.138516431108876),"332, West Meda Avenue, Glendora, Los Angeles C...",123399006,122858265,0,POINT (-117.75890605167689 34.0576222887382),"901, West 2nd Street, Edison Historic District...",...,17 mins,17 mins,17 mins,17 mins,17 mins,18 mins,17 mins,17 mins,17 mins,19.5 km
